In [1]:
import requests
import json
from bs4 import BeautifulSoup
import os

In [2]:
## change the
query    = " how to find marker genes for single cell RNA seq study "
question = " how to find marker genes for single cell RNA seq study "
print(type(query))

start_date = "2010-01-01" ## "YYYY-MM-DD"
end_date   = "2023-12-30"

# Replace with the actual bioRxiv API endpoint for searching articles
url = f"https://api.biorxiv.org/details/biorxiv/{start_date}/{end_date}"

# Parameters for the API request
params = {
    'q': query,  # your search query
    'num_results': 10  # number of results to return
}

# Send a request to the bioRxiv API
response = requests.get(url, params=params)




<class 'str'>


In [3]:
#  Use this if you want to return exacly 10 results from the api and not 100 and then filter the first 10 like in we 
#did above
'''
start_date = "2010-01-01"  # "YYYY-MM-DD"
end_date = "2023-12-30"
desired_num_results = 10  # number of results you want to return
articles = []

cursor = 0
while len(articles) < desired_num_results:
    url = f"https://api.biorxiv.org/details/biorxiv/{start_date}/{end_date}/{cursor}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data: {response.status_code}")
        break

    data = response.json()
    articles.extend(data['collection'])  # Assuming 'collection' contains the articles
    cursor += 100  # Increment the cursor to get the next page of results

    # Check if there are no more articles
    if len(data['collection']) < 100:
        break

# Slice the list to contain only the desired number of articles
articles = articles[:desired_num_results]
'''

'\nstart_date = "2010-01-01"  # "YYYY-MM-DD"\nend_date = "2023-12-30"\ndesired_num_results = 10  # number of results you want to return\narticles = []\n\ncursor = 0\nwhile len(articles) < desired_num_results:\n    url = f"https://api.biorxiv.org/details/biorxiv/{start_date}/{end_date}/{cursor}"\n    response = requests.get(url)\n    if response.status_code != 200:\n        print(f"Failed to retrieve data: {response.status_code}")\n        break\n\n    data = response.json()\n    articles.extend(data[\'collection\'])  # Assuming \'collection\' contains the articles\n    cursor += 100  # Increment the cursor to get the next page of results\n\n    # Check if there are no more articles\n    if len(data[\'collection\']) < 100:\n        break\n\n# Slice the list to contain only the desired number of articles\narticles = articles[:desired_num_results]\n'

In [4]:
# Check if the request was successful
if response.status_code == 200:
    # Parse the response JSON
    number_of_articles = 10  # Filter the first 10 articles
    data = response.json()

    # Extract article information
    articles = data['collection']  # Adjust this based on actual response structure
    articles = articles[:number_of_articles]

    # Create a directory for articles if it doesn't exist
    if not os.path.exists('articles'):
        os.makedirs('articles')

    for article in articles:
        #print(f"Title: {article['title']}")

        if article['published'] == "NA":
            print(f"Title: {article['title']}")

            doi = article['doi']
            # Replace / with _ in DOI to make it a valid filename
            filename = doi.replace('/', '_') + '.txt'
            # Include the folder name in the path
            filepath = os.path.join('articles', filename)
    
            # bioRxiv metadata
            full_text_url = f"https://www.biorxiv.org/content/{doi}.full"
    
            try:
                response = requests.get(full_text_url)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    texts = soup.get_text(separator=' ', strip=True)
    
                    # Writing the texts to a file inside the articles folder
                    with open(filepath, 'w', encoding='utf-8') as file:
                        file.write(texts)
                    print(f"Text written to {filepath}")
    
            except Exception as e:
                print(f"Problem reading the article: {e}")

Title: Population genomics of Saccharomyces cerevisiae human isolates: passengers, colonizers, invaders.
Text written to articles/10.1101_001891.txt
Title: Estimating seed bank accumulation and dynamics in three obligate-seeder Proteaceae species
Text written to articles/10.1101_001867.txt
Title: How and where to look for tRNAs in Metazoan mitochondrial genomes, and what you might find when you get there
Text written to articles/10.1101_001875.txt
Title: How and where to look for tRNAs in Metazoan mitochondrial genomes, and what you might find when you get there
Text written to articles/10.1101_001875.txt
Title: Emergence of structural and dynamical properties of ecological mutualistic networks
Text written to articles/10.1101_001818.txt


In the above the implemtation is to write them to text files, whether keeping the first article or many, for easier loading when saving to vectorstore

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader



In [6]:
# Load and process the text files
#loader = TextLoader('article.txt')

#Load from a directory
loader = DirectoryLoader('./articles/', glob="./*.txt", loader_cls=TextLoader) #- We will use this in case of many articles saved in a directory

documents = loader.load()

In [7]:
#splitting the text into smaller documents 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [8]:
# Lets see how many smaller documents we have created from the large document
len(texts)

278

In [9]:
# Lets select the 4th document to see how it looks as a small chunk
texts[50]

Document(page_content='would be only 6 nt (out of 61 nt) of overlap with the tRNA-ser(UCN) gene. Alternatively, the l-rRNA gene could be re-annotated to match more closely the CRW H4 stem structures, in which case there would be 19 nt of overlap (plus another ∼2 nt if the gene extends beyond the end of the stem). The latter seems to be the most likely option. Download figure Open in new tab Fig. 4 (a) Size of the gaps (number of nucleotides) between adjacent genes, and (b) amount of overlap of each tRNA (%) with another gene, in the Epiperipatus biolleyi (Onychophora) mitochondrial genome, based on the new annotations. Each dot represents (a) one boundary between genes and (b) one tRNA. Negative numbers indicate overlap between gene sequences. Not shown is the largest gap between genes (449 nucleotides), which is assumed to be the AT-rich or control region. The two databased Varroa destructor (Arthropoda; Chelicerata) mitogenomes were annotated to have different tRNAs completely overla

In [10]:
import os
from dotenv import load_dotenv
import chromadb
#from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings # use openai embeddings instead

#from langchain_community.vectorstores import Chroma  # We will use this as our local vectorstore


# Load environment variables from a .env file
load_dotenv()

OPENAI_API_KEY = os.getenv('openai_api_key')


# Embed and store the texts


## here we are using OpenAI embeddings but we could also use the sentensetransformer embeddings
embedding_function = OpenAIEmbeddings(model = 'text-embedding-ada-002',openai_api_key=OPENAI_API_KEY)

# create the open-source embedding function 
#embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(texts, embedding_function)


/home/acelogic/Jupyter Environs/langchain/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [38]:
# Making a retriever
def retriever():
    retriever = db.as_retriever(search_kwargs={"k": 20})
    docs = retriever.get_relevant_documents(query) # Here we are filtering documents with similar meaning to the query
    return docs
docs = retriever()
print(docs)

[Document(page_content='Department of Biomedical Sciences and Veterinary Public Health, Swedish University of Agricultural Sciences , Uppsala 751 89, Sweden ; Find this author on Google Scholar Find this author on PubMed Search for this author on this site For correspondence: David.Morrison@bvf.slu.se Tony D. Auld 3 Plant Ecology Unit, Climate Change Science Section, Department of Environment & Climate Change (NSW) , PO Box 1967, Hurstville NSW 2220, Australia ; Find this author on Google Scholar Find this author on PubMed Search for this author on this site For correspondence: Tony.Auld@environment.nsw.gov.au Abstract Full Text Info/History Metrics Preview PDF Abstract The seed bank dynamics of the three co-occurring obligate-seeder (i.e. fire-sensitive) Proteaceae species, Banksia ericifolia , Banksia marginata and Petrophile pulchella , were examined at sites of varying time since the most recent fire (i.e. plant age) in the Sydney region. Significant variation among species was fou

In [39]:
def print_sources(documents):
    seen_dois = set()  # A set to keep track of DOIs that have already been printed

    for doc in documents:
        
        # Extract the 'source' from the document's metadata
        source = doc.metadata['source']
        
        # Extract the DOI part from the 'source'
        parts = source.split('/')[1].split('.')  # Split into parts by '.' after 'articles/'
        doi = '.'.join(parts[:-1])  # Join all parts except the last one ('txt')

        # Replace underscore with slash in DOI
        doi = doi.replace('_', '/')

        # Print the DOI if it hasn't been seen before

        if doi not in seen_dois:
            print( f'Relevant Docs: Doi: {doi} ')
            seen_dois.add(doi)  # Add the DOI to the set of seen DOIs


print_sources(docs)


Relevant Docs: Doi: 10.1101/001867 
Relevant Docs: Doi: 10.1101/001891 


In [40]:
# Set your OpenAI API key here
OPENAI_API_KEY = os.getenv('openai_api_key')
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY
)


def summarize_query(query, documents):
    # Construct the conversation
    messages = [{"role": "system",
                 "content": """
                 You are a helpful research assistant.Summarize ONLY from the documents given to answer the question. 
                 Dont generate or add your own content. If there is no answer in the documents just say there is no
                 answer in the available documents. I insist dont generate your own content
                    """
                }]
    for doc in documents:
        content = doc.page_content if hasattr(doc, 'page_content') else ''
        messages.append({"role": "user", "content": content})
    messages.append({"role": "user", "content": query})

    # Make the API call
    try:
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=messages,
            temperature=0.7  # creativity
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None




In [42]:
#Testing with an excerpt from one of the documents
excerp = """Unfortunately, tRNA genes in Metazoan mitochondria have proved to be problematic because they often vary in number
(genes missing or duplicated) and also in the secondary structure of the transcribed tRNAs (T or D arms missing)
"""
query = 'Why have tRNA genes in Metazoan mitochondria have proved to be problematic'
docs = retriever()
summary = summarize_query(query, documents=docs)
print(summary)
print_sources(docs)

The tRNA genes in Metazoan mitochondria have proved to be problematic because they often vary in number, with some genes missing or duplicated, and also vary in the secondary structure of the transcribed tRNAs, including missing T or D arms. This variability makes it difficult to locate all of the genes within the mitogenome and to precisely annotate their 5′ and 3′ boundaries. The standard cloverleaf-shaped secondary structure may lack one of the cloverleaf arms, and the structure and number of tRNAs can differ both between tRNA types and between taxonomic groups. Additionally, this variability has led to a lack of understanding of how these mitochondrial tRNAs function compared to the more standardized nuclear tRNAs. Such problems with annotation and identification of tRNA genes result in missed genes, wrongly annotated locations, false complements, and inconsistent criteria for assigning the 5′ and 3′ boundaries in the databased mitogenome sequences.
Relevant Docs: Doi: 10.1101/0018

In [45]:
query =  "which year was Elon Musk born?"
docs = retriever()
summary = summarize_query(query, documents=docs)
print(summary)
print_sources(docs)

There is no answer in the available documents.
Relevant Docs: Doi: 10.1101/001867 
Relevant Docs: Doi: 10.1101/001891 


In [47]:
# Asking a new question
query =  " "  # and hit run                                                     
docs = retriever()
summary = summarize_query(query, documents=docs)
print(summary)
print_sources(docs)  # and hit run

There is no answer in the available documents.
Relevant Docs: Doi: 10.1101/001891 
Relevant Docs: Doi: 10.1101/001867 
Relevant Docs: Doi: 10.1101/001818 
Relevant Docs: Doi: 10.1101/001875 


In [16]:
# Write article to file
with open("article.html","w") as file:
  file.write(str(response.content))

In [17]:
# installs
#pip install langchain tiktoken openai langchainhub chromadb 